# Web Scraper for trip advisor

## Prerequisites

In [ ]:
pip install selenium

## Imports

In [1]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

## Constants

In [2]:
URL_RESTAURANTS = "https://www.tripadvisor.com/Restaurants-g189473-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"
URL_ONE_CAFE = "https://www.tripadvisor.com/Restaurant_Review-g189473-d3807291-Reviews-To_Tsai_Thessaloniki-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"

PATH = "chromedriver.exe"

## Custom tools

#### Elements Class - Contains all the element identifiers

In [3]:
class element:
    '''
    This is a class that contains all the elements we will need to scrape data from TripAdvisor
    '''
    COOKIES_ACCEPT_BUTTON = (By.ID, "onetrust-accept-btn-handler")
    SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON = (By.CLASS_NAME, "fdmYH")

    #Establishment Type elements:
    RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_3')
    RESTAURANTS_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")
    COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_6')
    COFFEE_AND_TEA_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")
    BARS_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_241')
    BARS_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{BARS_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")


    LIST_ITEM = (By.XPATH,".//div[@data-test='&1_list_item']")
    LIST_ITEM_URL = (By.XPATH,".//a[@class='bHGqj Cj b']")
    REVIEW_TABLE = (By.ID, "taplc_location_reviews_list_resp_rr_resp_0")
    REVIEWS_COUNT = (By.CLASS_NAME, "reviews_header_count")
    REVIEW_CONTAINER = (By.XPATH,".//div[@class='review-container']")
    EXPAND_REVIEW_BUTTON = (By.XPATH,"//span[@class='taLnk ulBlueLinks']")

    REVIEW_TITLE = (By.XPATH,".//span[@class='noQuotes']")
    REVIEW_DATE = (By.XPATH,".//span[contains(@class, 'ratingDate')]")
    REVIEW_RATING = (By.XPATH,".//span[contains(@class, 'ui_bubble_rating bubble_')]")
    REVIEW_TEXT = (By.XPATH,".//p[@class='partial_entry']")
    NEXT_PAGE_IN_REVIEWS = (By.XPATH,'.//a[@class="nav next ui_button primary"]')

#### Establishments Class - Used to along with the select_establishments method of TripAdvisorScrapper

In [4]:
class establishment:
    def __init__(self,checkboxElement,buttonElement,clickWhenStateIs):
        self.checkboxElement = checkboxElement
        self.buttonElement = buttonElement
        self.clickWhenStateIs = clickWhenStateIs

#### TripAdvisor Scraper Class

In [10]:
class TripAdvisorScraper:
    def __init__(self, path, url):
        '''
        Constructor
        '''
        self.driver = webdriver.Chrome(PATH)
        self.url = url

    def getElementObjects(self, element, rootItem = None,placeHolderValues:list=None):
        '''
        This method is used to gather all matching elements
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i),placeHolderValues[i])
        return rootItem.find_elements(element[0],value)

    def getElementObject(self, element, rootItem = None,placeHolderValues:list=None):
        '''
        This method is used to gather an element defined in the elements class
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i),placeHolderValues[i])
        return rootItem.find_element(element[0],value)

    def waitForElement(self,element,rootItem = None, placeHolderValues:list=None):
        '''
        This method is used to gather an element defined in the elements class but it also waits until the element is presented
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i+1),placeHolderValues[i])
        try:
            return WebDriverWait(rootItem, 20).until(EC.presence_of_element_located((element[0],value)))
        except:
            return None

    def acceptCookies(self):
        try:
            cookies = self.waitForElement(element.COOKIES_ACCEPT_BUTTON)
            cookies.click()
        except:
            print("No cookies! Impolite! Meh...")

    def open_browser(self):
        '''
        This method is used to navigate to the URL that was instructed to the scraper durring initialization
        '''
        self.driver.get(self.url) #Load URL
        self.acceptCookies()

    def select_establishments(self,establishments:list):
        '''
        This method is used to select establishment types on TripAdvisor
        Parameters:
        establistmentElements :
            Gets a list of objects with three keys, checkbox, initialState and button :
                'checkbox'      : The checkbox element to check (Type: element object from element class)
                'button'        : The button element to click in order to change the state (Type: element object from element class)
                'initialState'  : The initial state of the checkbox (Type: boolean, True/False)
        '''

        #Expand list of establishments:
        try:
            #there are other elements with the same class name, but this is the first one
            more_button = self.waitForElement(element.SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON)
            more_button.click()
        except:
            pass

        #Select establishment types:
        for e in establishments:
            if type(e) != establishment: continue
            try:
                establishment_checkbox = self.getElementObject(e.checkboxElement).is_selected()
                if establishment_checkbox == e.clickWhenStateIs:
                    self.getElementObject(e.buttonElement).click()
            except:
                pass

    def loop_at_rest(self):
        num_page = 2
        for i in range(0, num_page):
            num_items = 1
            for j in range(1, num_items+1):
                time.sleep(10)
                item = self.waitForElement(element.LIST_ITEM, placeHolderValues=[str(j)] )
                url = self.getElementObject(element.LIST_ITEM_URL, rootItem=item).get_attribute("href")
                self.driver.get(url)
                self.get_reviews()

    def get_reviews(self):
        self.acceptCookies()
        #num_page = 10
        review_table = self.waitForElement(element.REVIEW_TABLE)
        try:
            reviews_no = self.getElementObject(element.REVIEWS_COUNT).text
            reviews_no = int(reviews_no.replace("(", "").replace(")", ""))
            num_page = int(reviews_no / 10)
            print("page = ",num_page)
            k=0
            for i in range(0, num_page):
                # expand the review
                time.sleep(4) #<= We need to think about this waiting time {!}
                container = self.getElementObjects(element.REVIEW_CONTAINER)
                print("container ", len(container))
                for j in range(len(container)):
                    try:
                        try:
                            time.sleep(2) #<= We need to think about this waiting time {!}
                            more = self.getElementObject(element.EXPAND_REVIEW_BUTTON,rootItem=container[j])
                            if more != None:
                                if more.text == "More":
                                    more.click()
                        except StaleElementReferenceException:
                            print("Comment ",j, " could be longer, but I will let that pass..." )

                        title = self.getElementObject(element.REVIEW_TITLE,rootItem=container[j]).text
                        date = self.getElementObject(element.REVIEW_DATE,rootItem=container[j]).get_attribute("title")
                        rating = self.getElementObject(element.REVIEW_RATING,rootItem=container[j]).get_attribute("class").split("_")[3]
                        review = self.getElementObject(element.REVIEW_TEXT,rootItem=container[j]).text.replace("\n", " ")
                        print(title)
                        print(date)
                        print(rating)
                        print(review)
                        print("-----------")
                    except NoSuchElementException:
                        print("Didn't found item ",j, ", but it should be there.")
                k=i
                # change the page
                self.getElementObject(element.NEXT_PAGE_IN_REVIEWS).click()
        except NoSuchElementException:
            print("Oops, unlucky search!")
        except TimeoutException:
            print("You run out of time!")

        time.sleep(2)
        self.driver.execute_script("window.history.go(-1)") #go back

    def quit_browser(self):
        self.driver.quit()


In [ ]:
TripAdvisorScraper = TripAdvisorScraper(PATH,URL_RESTAURANTS)
TripAdvisorScraper.open_browser()
TripAdvisorScraper.select_establishments(
    establishments=[
        establishment(element.RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX,element.RESTAURANTS_ESTABLISHMENT_TYPE_BUTTON,True),
        establishment(element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX,element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_BUTTON,False),
        establishment(element.BARS_ESTABLISHMENT_TYPE_CHECKBOX,element.BARS_ESTABLISHMENT_TYPE_BUTTON,False)
    ]
)
TripAdvisorScraper.loop_at_rest()
# TripAdvisorScraper.get_reviews()
TripAdvisorScraper.quit_browser()

<ipython-input-10-c630862cfc5e>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(PATH)


page =  19
container  10
Delicious tea, for tea lovers
December 19, 2021
50
If you are a tea lover , this is the place to visit in Thessaloniki. Great variety, nice atmosphere, perfect service.
-----------
Awesome place!
June 20, 2021
50
The staff was amazing, the tea sublime and the ambience great! I got the yerba mate and then came back the next day for an iced vanilla matcha latte. Both teas were so good I didn't feel like doing anything except drinking and enjoying the experience
-----------
Detox
June 19, 2020
50
To Tsai is a wonderful place, calm, quiet, relaxing music, very friendly staff, great variety of teas - warm recommendations!
-----------
Very good experience
January 3, 2020
50
It is a non smoking shop. The Professionel is very polite. In general it's a very good place to visit with friends family or alone.
-----------
Best break
July 6, 2019
50
Any kind of tea, hot or cold, all delicious. Heat survival iced tea (I recommend you ask for regular strong, black, with lemon 

Best tea in Greece
July 22, 2016
50
One of the best shops in which you can have your tea (obviously) coffee or chocolate in Thessaloniki! Many flavors, many different sweets and toasts, always fresh and baked the moment you order them. My favorite tea taste is the "White Angel". Also very tasty the "fire in kamini" especially during winter.
-----------
Breakfast
July 21, 2016
50
Came hear on the recommendation from trip adviser So glad we did, service great, food excellent and loved the whole experience of pouring the tea Great start to the day Totally Will recommend it Chico
-----------
container  10
Unique place
June 8, 2016
50
What a gem! Congrats to the people who made this a reality in Salonica. In the middle of the city but away from the noise. The service is excellent, the staff really eager to help and explain. A large selection of tea choices that can...More
-----------
A haven
May 16, 2016
50
In the city of a thousand and one cafes, this place caught my eye because although c

container  10
A cup of tea
January 6, 2015
40
With a very looong catalog for tea it's very difficult for me to decide which one would please me best. Luckily the staff will come to my aid and I will be advised properly.
-----------
Tea paradise
December 18, 2014
50
If you do like tea, you must visit this place. You can choose between a variety of both rich in flavor and smell teas. Green tea, black tea what ever you like. They have so many combinations. The staff is so helpful and eager to show you all the tea jars until you choose the one you like most. I had the "Fotia sto kamini" which was a kind of sweet and spicy at the same time.
-----------
Good choice
November 26, 2014
50
For the tea lovers, especialy but not specificaly, this is the place to go! Cosy place with friendly staff!
-----------
Lovely experience
October 7, 2014
50
Read the previous reviews and I agree with them all.I had Greek mountain tea which had big buds and stalks, herbal and fresh.Only small seating area outsi

Amazing food experience!
October 15, 2021
50
My first day in Thessaloniki I had the greatest welcome at your place! Truly amazing food and amazing people. THANK YOU! Totally recommend !
-----------
Great hole in the wall food!
October 10, 2021
50
Argofageio, or Argophageio on Google maps, or Αργοφαγειο on the restaurant's signage is a great gem in a small place. I mentioned the names as I had a hard time telling the name to a taxi driver. I eventually used Uber app to book a taxi, so I didn't have to explain where I was going. When we got the the restaurant's name was in Greek, so I was afraid the taxi dropped us in an alley. I open the door, went down a couple of steps. They can seat a maximum of 15 people and only 2 customers were there finishing their meal. I thought I was in the wrong place, but then the waiter said my name.  Niko, the waiter, asked for our EU covid vaccine QR codes, but we are from the US, so we had our CDC cards. I said I called ahead and was told that our US car

Excellent traditional greek restaurant
December 20, 2020
50
Small, clean and friendly, traditional Greek restaurant with great dishes and super friendly service. The cook came twice and asked if everything was ok. My partner paid but the prices were excellent, at least from what I could see on the menu. Would definitely visit again.
-----------
The best for comfort food made with love.
December 18, 2020
50
It was a dear friend who invited me there to meet "a great chef" as he mentioned. He wasn't lying. We had personal assistance by the chef himself who proposed and offered us one of the best meals I had in the city. The wine was plenty and of a great quality too. I surely recommend it to any visitor or passerby.
-----------
A hidden gem!
November 22, 2020
50
This is a small hidden gem within Thessaloniki center. The owner gets his ingredients from different parts of Greece, changes the menu frequently and innovates new recipes. He prepared a table for us, every single dish was out of 

Fabulous food, lovely taverna
January 18, 2020
50
Of the 4 restaurants we dined at during our stay in Thessaloniki, this was my favourite. Located in a small side street, Argofageio is everything we could have asked for. Service is friendly and personal, dishes are explained if required and the chef came out to ask if we were enjoying the meal. I had the fresh artichokes as an appetiser, they were the best I've ever had, followed by an enormous smoked pork chop. My husband had the feta and tomato to start followed by a chicken dish. Both meals were delicious, as was the local wine. We would certainly recommend this place and it deserves it's Trip Advisor high rating. Would advise to book as it gets busy.
-----------
Small restaurant, tasty food
January 15, 2020
40
In the center of the city , this is a small taverna with few tables , probably you will need to make a reservation to eat. The dishes were very good especially the ' kopsidia' (chops) which were delicious.Prices are reasonabl

Hidden Gem in Thessaloniki
October 20, 2019
50
The reviews from TA inspired our visit for Argofageio for lunch, we were looking for it on google maps but couldn’t find it...we walked back down the street and then came across it!  Don’t be put off by the uncommercial exterior, this little restaurant is serving top quality traditional Greek food! Our waiter Nick was so friendly and was also very helpful when we were choosing a selection of dishes from the menu. The food is fresh and seriously flavorsome! We had beautiful dishes like Feta Saganaki, Dolmades (stuffed vine leaves) and baked aubergines to name a few! The portion sizes are good for all of the dishes, which we had served with lovely warm bread. We also had ordered the house white wine which was lovely. This restaurant is excellent value for money and well worth a visit!
-----------
Amazing!
October 20, 2019
50
An amazing place! Personal, fantastic atmosphere, mix of tastes, even beyond the main street tsatsiki! Dont even think